In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import *
import time
from optparse import OptionParser
import matplotlib
from matplotlib import pyplot as plt
from tikzplotlib import save as tikz_save

In [2]:
#Compressive_Strength
df = pd.read_csv('../Dataset/compressive_strength.csv')
df.head()

,Cement (kg in a m^3 mixture),Blast Furnace Slag (kg in a m^3 mixture),Fly Ash (kg in a m^3 mixture),Water (kg in a m^3 mixture),Superplasticizer (kg in a m^3 mixture),Coarse Aggregate (kg in a m^3 mixture),Fine Aggregate (kg in a m^3 mixture),Age (day),"Concrete compressive strength (MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [3]:
#Tensile_Strength
dp = pd.read_csv('../Dataset/tensile_strength.csv')
dp.head()

,Compressive strength of cement fce (MPa),Tensile strength of cement fct (MPa),Curing age (day),Dmax of crushed stone (mm),Stone powder content in sand (%),Fineness modulus of sand,W/B,"Water to cement ratio, mw/mc",Water (kg/m3),Sand ratio (%),Slump (mm),"Compressive strength, fcu,t (MPa)","Splitting tensile strength, fst,t (MPa)"
0,46.8,8.0,3,31.5,5.0,3.34,0.56,0.56,180.0,44.0,50.0,32.5,1.18
1,46.8,8.0,3,31.5,9.0,3.27,0.56,0.56,180.0,44.0,70.0,28.7,1.13
2,46.8,8.0,3,31.5,13.0,2.77,0.56,0.56,180.0,44.0,50.0,28.5,1.56
3,46.8,8.0,7,31.5,5.0,3.34,0.56,0.56,180.0,44.0,50.0,33.5,1.39
4,46.8,8.0,7,31.5,9.0,3.27,0.56,0.56,180.0,44.0,70.0,34.9,1.38


In [4]:
#Tensile_strength_data2set
dy = pd.read_csv('../Dataset/data2set.csv')
dy.head()

,Curing age (day),"Compressive strength, fcu,t (MPa)","Splitting tensile strength, fst,t (MPa)"
0,3,32.5,1.18
1,3,28.7,1.13
2,3,28.5,1.56
3,7,33.5,1.39
4,7,34.9,1.38


In [5]:
def mean_absolute_percentage_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [6]:
# cross validation

def cv(random_state, poly_degree, model, problem, print_folds=True):
    interaction_only = False
    
    if problem.lower() == "compressive":
        data_file ='../Dataset/compressive_strength.csv'
        data = pd.read_csv(data_file)
        interaction_only = True

    elif problem.lower() == "tensile":
        data_file ='../Dataset/tensile_strength.csv'
        data = pd.read_csv(data_file)
        
    elif problem.lower() == "test2":
        data_file ='../Dataset/data2set.csv'
        data = pd.read_csv(data_file)

    else:
        print("The problem has to be compressive or tensile")
        return

    data = data.values
    n_data_cols = np.shape(data)[1]
    n_features = n_data_cols - 1

    # retrieve data for features
    X = np.array(data[:, :n_features])
    y = np.array(data[:, n_features:])
    # split into 10 folds with shuffle
    n_folds = 10
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)

    start_time = time.time()
    scores = []
    fold_index = 0

    for train_index, test_index in kf.split(X):
        X_train = X[train_index]
        y_train = y[train_index]
        X_test = X[test_index]
        y_test = y[test_index]

        X_scaler = MinMaxScaler(feature_range=(0, 1))
        X_train = X_scaler.fit_transform(X_train)
        X_test = X_scaler.transform(X_test)

        y_scaler = MinMaxScaler(feature_range=(0, 1))
        y_train = y_scaler.fit_transform(y_train)

        if poly_degree >= 1:
            poly = PolynomialFeatures(degree=poly_degree, interaction_only=interaction_only)
            X_train = poly.fit_transform(X_train)
            X_test = poly.transform(X_test)
            # print ('Total number of features: ', X_train.size)

        model.fit(X_train, y_train.ravel())

        y_pred = model.predict(X_test)
        y_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))

        # y_train_pred = model.predict(X_train)
        # y_train_pred = y_scaler.inverse_transform(y_train_pred.reshape(-1, 1))

        # y_train = y_scaler.inverse_transform(y_train)

        # Error measurements
        r_lcc = r2_score(y_test, y_pred) ** 0.5
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        # print("RMSE on train: %s" % mean_squared_error(y_train, y_train_pred) ** 0.5)
        # print("RMSE on test: %s" % rmse)
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        scores.append([r_lcc, rmse, mae, mape])
        #if print_folds:
           # print("[fold {0}] r: {1:.5f}, rmse(MPa): {2:.5f}, mae(MPa): {3:.5f}, mape(%): {4:.5f}".
                  #format(fold_index, scores[fold_index][0], scores[fold_index][1], scores[fold_index][2], scores[fold_index][3]))
        fold_index += 1
    scores = np.array(scores)
    # barplot(["R2", "RMSE", "MAE", "MAPE"], scores.mean(0), scores.std(0), "Metrics", "Values",
    #         "Performance with different metrics")
    print('k-fold mean:              ', scores.mean(0))
    print('k-fold standard deviation:', scores.std(0))

    # Running time
    print('Running time: %.3fs ' % (time.time() - start_time))
    return scores.mean(0)[1].item()


In [7]:
def run_model(regressor, params, random_state, poly_degree, problem):
    model = regressor(**params)
    cv(random_state=random_state, poly_degree=poly_degree, model=model, problem=problem)

In [8]:
from sklearn.neural_network import MLPRegressor

# run cross validation for MLP
def run_mlp(random_state, poly_degree, hd_layer_1, hd_layer_2, solver, max_iter, alpha, problem):
    
    print("Running MLP for %s data with "
          "random_state=%s, poly_degree=%s, hd_layer_1=%s, hd_layer_2=%s, solver=%s, max_iter=%s, alpha=%s" %
          (problem, random_state, poly_degree, hd_layer_1, hd_layer_2, solver, max_iter, alpha))
    hd_layers = (hd_layer_1, hd_layer_2, )
    if hd_layer_2 < 0:
        hd_layers = (hd_layer_1, )

    model = MLPRegressor(warm_start=False, random_state=random_state,
                         hidden_layer_sizes=hd_layers, solver=solver, max_iter=max_iter, alpha=alpha)
    
    cv(random_state, poly_degree, model, problem=problem)
    print("Finished running MLP for %s data with "
          "random_state=%s, poly_degree=%s, hd_layer_1=%s, hd_layer_2=%s, solver=%s, max_iter=%s, alpha=%s" %
          (problem, random_state, poly_degree, hd_layer_1, hd_layer_2, solver, max_iter, alpha))


In [9]:
#def random_search():
    
    #problem
    #problem_opts = np.array(["compressive","tensile","test2"])
    
    #solver
    #solver_opts = np.array(["lbfgs","sgd"])
    
    #alpha 
    #alpha_opts = np.array([0,0.0001])
    
    # random_state
    #random_state_opts = np.array([0, 1, 2, 42]) 
    
    # poly_degree
    #poly_degree_opts = np.arange(1, 5, 1)  # 4

    # hd layer size 1
    #hd_layer_1_opts = np.array([100, 200, 300, 400, 500, 1000, 2000])  # 7

    # hd layer size 2
    #hd_layer_2_opts = np.array([-1, 100, 200, 300, 400, 500])  # 6

    # max iter
    #max_iter_opts = np.array([100, 200, 300, 400, 500, 1000])  # 6

    #for i in range(20):
        #hd_layer_1 = np.random.choice(hd_layer_1_opts)

        #hd_layer_2 = np.random.choice(hd_layer_2_opts)

        #max_iter = np.random.choice(max_iter_opts)
        
        #random_state = np.random.choice(random_state_opts)
        
        #poly_degree = np.random.choice(poly_degree_opts)
        
        #alpha = np.random.choice(alpha_opts)
        
        #solver = np.random.choice(solver_opts)
        
        #problem = np.random.choice(problem_opts)

        
        #run_mlp(random_state, poly_degree, hd_layer_1, hd_layer_2, solver, max_iter, alpha, problem)


#if __name__ == "__main__":
    #random_search()

In [10]:
#compressive
if __name__ == "__main__":
    run_mlp(random_state=0, poly_degree=3, hd_layer_1=300, hd_layer_2=100, solver="lbfgs", max_iter=1000, 
            alpha=0, problem="compressive")

Running MLP for compressive data with random_state=0, poly_degree=3, hd_layer_1=300, hd_layer_2=100, solver=lbfgs, max_iter=1000, alpha=0


C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


k-fold mean:               [0.96208959 4.30755798 2.92446948 9.80575871]
k-fold standard deviation: [0.01027563 0.51577215 0.26447208 0.83603682]
Running time: 882.956s 
Finished running MLP for compressive data with random_state=0, poly_degree=3, hd_layer_1=300, hd_layer_2=100, solver=lbfgs, max_iter=1000, alpha=0


In [11]:
#data2set
if __name__ == "__main__":
    run_mlp(random_state=0, poly_degree=3, hd_layer_1=100, hd_layer_2=200, solver="lbfgs", max_iter=1000, 
            alpha=0.0001, problem="test2")

Running MLP for test2 data with random_state=0, poly_degree=3, hd_layer_1=100, hd_layer_2=200, solver=lbfgs, max_iter=1000, alpha=0.0001


C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERAT

k-fold mean:               [ 0.95791396  0.3820856   0.27057535 10.1783207 ]
k-fold standard deviation: [0.00839331 0.03955436 0.0264372  0.92942203]
Running time: 315.811s 
Finished running MLP for test2 data with random_state=0, poly_degree=3, hd_layer_1=100, hd_layer_2=200, solver=lbfgs, max_iter=1000, alpha=0.0001


In [12]:
#tensile
if __name__ == "__main__":
    run_mlp(random_state=0, poly_degree=2, hd_layer_1=300, hd_layer_2=300, solver="lbfgs", max_iter=200, 
            alpha=0.0001, problem="tensile")

Running MLP for tensile data with random_state=0, poly_degree=2, hd_layer_1=300, hd_layer_2=300, solver=lbfgs, max_iter=200, alpha=0.0001


C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERAT

k-fold mean:               [0.97725625 0.28052823 0.19531032 7.97766254]
k-fold standard deviation: [0.00710043 0.03676206 0.01907213 1.07417921]
Running time: 328.598s 
Finished running MLP for tensile data with random_state=0, poly_degree=2, hd_layer_1=300, hd_layer_2=300, solver=lbfgs, max_iter=200, alpha=0.0001


C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
